In [29]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import pytz
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from concurrent.futures import ThreadPoolExecutor, as_completed
import itertools
import logging
import time

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class MarketEnvironment:
    def __init__(self):
        self.us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
        self.closest_time_index = None

    def adjust_to_trading_hours(self, dt):
        if dt.weekday() >= 5:  # Adjust for weekends
            dt += self.us_bd
        if dt.hour < 9 or (dt.hour == 9 and dt.minute < 30):
            dt = dt.replace(hour=9, minute=30)
        elif dt.hour > 16:
            dt = (dt + self.us_bd).replace(hour=9, minute=30)
        if dt.tzinfo is None or dt.tzinfo.utcoffset(dt) is None:
            dt = pytz.timezone('America/New_York').localize(dt)
        return dt

    def fetch_market_data(self, ticker, signal_date):
        signal_date = self.adjust_to_trading_hours(signal_date.tz_localize(pytz.timezone('America/New_York')))
        start_date_utc = (signal_date - timedelta(days=1)).astimezone(pytz.utc)
        end_date_utc = (signal_date + timedelta(days=6)).astimezone(pytz.utc)
        attempts, wait = 20, 0.001
        for attempt in range(attempts):
            try:
                data = yf.download(ticker, start=start_date_utc.strftime('%Y-%m-%d'), end=end_date_utc.strftime('%Y-%m-%d'), interval='1m', progress=False)
                if data.empty:
                    continue
                data.index = data.index.tz_convert(pytz.timezone('America/New_York'))

                self.closest_time_index = data.index.get_loc(signal_date, method='nearest')
                callout_price = data.at[data.index[self.closest_time_index], 'Close']
                
                # Split the data
                data_for_atr = data.loc[:data.index[self.closest_time_index]]
                data_for_backtest = data.loc[data.index[self.closest_time_index]:]
                
                return data_for_atr, data_for_backtest, callout_price
            except Exception as e:
                time.sleep(wait)
                wait *= 2  # Exponential backoff
        logging.error(f"Failed to download data for {ticker} after {attempts} attempts.")
        return None, None, None

    def calculate_atr(self, data, period):
        high_low = data['High'] - data['Low']
        high_close = np.abs(data['High'] - data['Close'].shift())
        low_close = np.abs(data['Low'] - data['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        return np.max(ranges, axis=1).rolling(window=period).mean()


class Backtester:
    def __init__(self, initial_capital=10000):
        self.initial_capital = initial_capital
        self.market_env = MarketEnvironment()

    def run_backtest(self, data_for_atr, data_for_backtest, callout_price, atr_multiplier, trailing_stop_multiplier, atr_period):
        atr = self.market_env.calculate_atr(data_for_atr, atr_period).iloc[-1] * atr_multiplier
        profit_losses = ((data_for_backtest['Close'] - callout_price) / callout_price * 100).tolist()
        return self.evaluate_trades(profit_losses, atr, trailing_stop_multiplier)

    def evaluate_trades(self, profit_losses, atr, trailing_stop_multiplier):
        portfolio = {
            'Capital': self.initial_capital,
            'Cash': self.initial_capital,
            'Equity': 0,
            'Returns': [],
            'Drawdowns': [],
            'Successful Trades': 0
        }

        initial_investment = portfolio['Cash']
        portfolio['Cash'] -= initial_investment
        portfolio['Equity'] = initial_investment
        max_profit_loss = 0
        minutes_taken = 0

        for i, profit_loss in enumerate(profit_losses):
            if profit_loss > max_profit_loss:
                max_profit_loss = profit_loss

            if profit_loss < max_profit_loss - (atr * trailing_stop_multiplier) or i == len(profit_losses) - 1:
                sell_amount = portfolio['Equity']
                drawdown = ((1 + max_profit_loss / 100) / (1 + profit_loss / 100)) - 1
                portfolio['Drawdowns'].append(drawdown)
                portfolio['Cash'] += sell_amount * (1 + profit_loss / 100)
                portfolio['Equity'] -= sell_amount
                portfolio['Returns'].append(profit_loss / 100)
                minutes_taken = i
                if profit_loss > 1 and drawdown < 0.005:
                    portfolio['Successful Trades'] += 1
                break

        total_return = (portfolio['Cash'] - self.initial_capital) / self.initial_capital
        portfolio_variance = np.var(portfolio['Returns']) if portfolio['Returns'] else 0
        sharpe_ratio = total_return / np.sqrt(portfolio_variance) if portfolio_variance else 0
        max_drawdown = max(portfolio['Drawdowns']) if portfolio['Drawdowns'] else 0
        avg_trade_gain = np.mean(portfolio['Returns']) if portfolio['Returns'] else 0

        return {
            'Total Return': total_return,
            'Portfolio Variance': portfolio_variance,
            'Sharpe Ratio': sharpe_ratio,
            'Final Equity': portfolio['Cash'],
            'Maximum Drawdown': max_drawdown,
            'Average Trade Gain': avg_trade_gain,
            'Successful Trades': portfolio['Successful Trades'],
            'Minutes Taken': minutes_taken
        }


def optimize_strategy(ticker, created_at, data_for_atr, data_for_backtest, callout_price, param_ranges, backtester):
    results = []
    for atr_mult, stop_mult, atr_period in itertools.product(param_ranges['atr_multiplier'], param_ranges['trailing_stop_loss_multiplier'], param_ranges['atr_periods']):
        result = backtester.run_backtest(data_for_atr, data_for_backtest, callout_price, atr_mult, stop_mult, atr_period)
        if result:
            results.append({
                'Ticker': ticker,
                'created_at': created_at,
                'ATR Multiplier': atr_mult,
                'Trailing Stop Multiplier': stop_mult,
                'ATR Period': atr_period,
                'Total Return': result['Total Return'],
                'Portfolio Variance': result['Portfolio Variance'],
                'Sharpe Ratio': result['Sharpe Ratio'],
                'Final Equity': result['Final Equity'],
                'Maximum Drawdown': result['Maximum Drawdown'],
                'Successful Trades': result['Successful Trades'],
                'Minutes Taken': result['Minutes Taken'],
                'Score':(result['Total Return'] - result['Maximum Drawdown'])/(result['Minutes Taken'] + 0.0001)*6000
            })
    return results

def parallel_optimize_strategy(df, param_ranges):
    backtester = Backtester()
    results = []

    def process_row(row):
        data_for_atr, data_for_backtest, callout_price = backtester.market_env.fetch_market_data(row['Ticker'], pd.to_datetime(row['created_at']))
        if data_for_atr is not None and data_for_backtest is not None:
            return optimize_strategy(row['Ticker'], row['created_at'], data_for_atr, data_for_backtest, callout_price, param_ranges, backtester)
        else:
            return []

    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = {executor.submit(process_row, row): row for _, row in df.iterrows()}
        for future in as_completed(futures):
            result = future.result()
            if result:
                results.extend(result)
    
    return pd.DataFrame(results)


def return_stock(message):
    if len(message.split()) > 1:
        return message.split()[0].replace('[','').replace(']','').replace('$', '').replace('\\', '').replace('*','').upper()

def return_buy_sell(message):
    if len(message.split()) > 1:
        return 1 if 'Buy' in message.split()[1].replace('[','').replace(']','') else 0

if __name__ == "__main__":
    df = pd.read_excel(r"C:\Users\amoog\Desktop\Project_X\Project_X\tweet_buy_or_sell.xlsx")
    df['Ticker'] = df['buy_or_sell'].apply(return_stock)
    df['Buy'] = df['buy_or_sell'].apply(return_buy_sell)
    df = df[(df['Ticker'].notnull()) & (df['Buy'] == 1)]
    param_ranges = {
        'atr_multiplier': np.arange(1, 3.5, 0.5),
        'trailing_stop_loss_multiplier': np.arange(1, 3.5, 0.5),
        'atr_periods': [14, 50, 100, 200, 400, 650]
    }
    results = parallel_optimize_strategy(df, param_ranges)
    best_results = results.loc[results.groupby(['Ticker', 'created_at'])['Total Return'].idxmax()]
    display(best_results)



1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted

- VVIX: No data found, symbol may be delisted

1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted



1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted

- VVIX: No data found, symbol may be delisted

1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted



1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted

- VVIX: No data found, symbol may be delisted

1 Failed download:
1 Failed downl

2024-05-24 10:52:27,219 - ERROR - Failed to download data for VVIX after 20 attempts.



1 Failed download:
- VVIX: No data found, symbol may be delisted


,Ticker,created_at,ATR Multiplier,Trailing Stop Multiplier,ATR Period,Total Return,Portfolio Variance,Sharpe Ratio,Final Equity,Maximum Drawdown,Successful Trades,Minutes Taken,Score
3595,AMD,2024-05-02 15:42:35.000000,3.0,3.0,50,0.058977,0.0,0,10589.766442,0.016300,0,1040,0.246211
1050,BITX,2024-05-09 12:42:17.000000,1.0,1.0,14,-0.002117,0.0,0,9978.825866,0.002122,0,1,-25.433375
600,BITX,2024-05-09 13:41:16.000000,1.0,1.0,14,0.001572,0.0,0,10015.723630,0.002875,0,7,-1.116869
3004,FXI,2024-05-06 13:36:45.999999,1.0,1.0,400,0.016844,0.0,0,10168.436837,0.006842,0,1702,0.035259
1205,FXI,2024-05-08 13:47:11.000000,1.0,1.0,650,0.058973,0.0,0,10589.733049,0.002820,1,1301,0.258970
904,FXI,2024-05-09 03:32:37.000000,1.0,1.0,400,0.024140,0.0,0,10241.404478,0.016071,0,1556,0.031114
3747,GOOG,2024-04-30 13:50:10.000000,3.0,3.0,200,0.004245,0.0,0,10042.445698,0.011571,0,499,-0.088090
2850,HOOD,2024-05-06 13:37:26.000001,1.0,1.0,14,-0.000537,0.0,0,9994.626540,0.000711,0,2,-3.745512
755,HOOD,2024-05-09 12:46:24.000000,1.0,1.0,650,0.037165,0.0,0,10371.651095,0.036389,0,1363,0.003417
3264,INTC,2024-05-05 19:08:31.000000,2.5,3.0,14,0.004815,0.0,0,10048.146241,0.003514,0,52,0.150091
